In [1]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [2]:
def gather_data(env):
	num_trials = 10000
	min_score = 50
	sim_steps = 500
	trainingX, trainingY = [], []

	scores = []
	for _ in range(num_trials):
		observation = env.reset()
		score = 0
		training_sampleX, training_sampleY = [], []
		for step in range(sim_steps):
			# action corresponds to the previous observation so record before step
			action = np.random.randint(0, 2)
			one_hot_action = np.zeros(2)
			one_hot_action[action] = 1
			training_sampleX.append(observation)
			training_sampleY.append(one_hot_action)
			
			observation, reward, done, _ = env.step(action)
			score += reward
			if done:
				break
		if score > min_score:
			scores.append(score)
			trainingX += training_sampleX
			trainingY += training_sampleY

	trainingX, trainingY = np.array(trainingX), np.array(trainingY)
	print("Average: {}".format(np.mean(scores)))
	print("Median: {}".format(np.median(scores)))
	return trainingX, trainingY

In [3]:
def create_model():
    model = Sequential()
    model.add(Dense(128, input_shape=(4,), activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(2, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"])
    return model

In [4]:
def predict():
    env = gym.make("CartPole-v0")
    trainingX, trainingY = gather_data(env)
#     print(trainingX.shape,trainingY.shape)
    model = create_model()
    model.fit(trainingX, trainingY, epochs=5)
    scores = []
    num_trials = 50
    sim_steps = 500
    for _ in range(num_trials):
        observation = env.reset()
        score = 0
        for step in range(sim_steps):
            action = np.argmax(model.predict(observation.reshape(1,4)))
#             print(action.shape)
            observation, reward, done, _ = env.step(action)
            score += reward
            if done:
                break
        scores.append(score)
    print(np.mean(scores))

if __name__ == "__main__":
    predict()

Average: 62.027692307692305
Median: 59.0
(20159, 4) (20159, 2)
Epoch 1/5
20159/20159 [==============================] - 11s - loss: 0.6893 - acc: 0.5532    
Epoch 2/5
20159/20159 [==============================] - 11s - loss: 0.6812 - acc: 0.5843    
Epoch 3/5
20159/20159 [==============================] - 9s - loss: 0.6780 - acc: 0.5837     
Epoch 4/5
20159/20159 [==============================] - 9s - loss: 0.6756 - acc: 0.5938     
Epoch 5/5
20159/20159 [==============================] - 9s - loss: 0.6744 - acc: 0.5970     
187.44
